In [1]:
import sys
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import glob
import plotly.express as px
%matplotlib inline

## Path

In [2]:
parent_dir = os.path.abspath(r'D:')
raw_data_dir = os.path.join(parent_dir, 'raw')
processed_data_dir = os.path.join(parent_dir, 'processed_data')
concatenated_yaws_dir = os.path.join(parent_dir, 'Concatenated_Yaws')


## List of files at dir

In [8]:

dir_path =parent_dir

items = os.listdir(dir_path)

# Filter out only the directories
folders = [item for item in items if os.path.isdir(os.path.join(dir_path, item))]

for folder_name in folders:
    folder_path = os.path.join(dir_path, folder_name)
    os.chdir(folder_path)
    all_files = glob.glob('*int')
    print(f"Processing {len(all_files)} files in folder: {folder_name}")
    print(all_files)


Processing 1892 files in folder: Yaw01
['0002.int', '0004.int', '0005.int', '0009.int', '0010.int', '0012.int', '0013.int', '0016.int', '0017.int', '0023.int', '0027.int', '0030.int', '0031.int', '0032.int', '0033.int', '0034.int', '0035.int', '0044.int', '0045.int', '0048.int', '0049.int', '0050.int', '0052.int', '0053.int', '0054.int', '0057.int', '0059.int', '0060.int', '0061.int', '0064.int', '0065.int', '0071.int', '0072.int', '0075.int', '0076.int', '0077.int', '0080.int', '0081.int', '0082.int', '0083.int', '0084.int', '0085.int', '0087.int', '0088.int', '0089.int', '0094.int', '0095.int', '0096.int', '0097.int', '0099.int', '0101.int', '0102.int', '0103.int', '0104.int', '0106.int', '0108.int', '0109.int', '0110.int', '0113.int', '0114.int', '0115.int', '0117.int', '0119.int', '0120.int', '0121.int', '0123.int', '0125.int', '0126.int', '0127.int', '0129.int', '0131.int', '0132.int', '0137.int', '0138.int', '0139.int', '0141.int', '0142.int', '0143.int', '0146.int', '0147.int', 


dir_path =raw_data_dir

items = os.listdir(dir_path)

# Filter out only the directories
folders = [item for item in items if os.path.isdir(os.path.join(dir_path, item))]

for folder_name in folders:
    folder_path = os.path.join(dir_path, folder_name)
    os.chdir(folder_path)
    all_files = glob.glob('*int')
    print(f"Processing {len(all_files)} files in folder: {folder_name}")
    print(all_files)


## Read binary data

In [4]:

sensor = (1,4,5,6,7,8,9,10,11,12,53,54,56,57,59,60,61,62,63,64)



def read_int(FileName,x):

    #print(FileName)

    xlen = len(FileName)
    ending = FileName[xlen-4:xlen]

    if ending=='.int':
        FileName = FileName[0:xlen-4]


    with open(FileName+ending,"rb") as f:
        f.read()
        filesize = f.tell()
       # print(filesize)
       

    f.close()




    with open(FileName+ending,"rb") as f:

    # the first 4 junk buffer
        
        leer = np.frombuffer(f.read(4),dtype=np.single)
        #print(leer)
    # the  24 buffer give information about file
        datei = np.frombuffer(f.read(24),dtype=np.uint32)
        #print(datei)

        f.seek(76)
    # read the number of sensor in file 
        lvek = np.frombuffer(f.read(4),dtype=np.uint32)
        #print(lvek)
     
    # then 4 times the number of sensors are skipped to read a control parameter:
  
        f.seek(84+lvek[0]*4)

        filetype = np.frombuffer(f.read(4),dtype=np.uint)
        #print(filetype)

        if filetype != 12:

    # start time is read (is not equal to 0 as we have been cut off from the simulation for the first 50 s
            t0 = np.frombuffer(f.read(4),dtype=np.single)
            #print(t0)

    # comes the parameter for the step size of the simulation

            dt = np.frombuffer(f.read(4),dtype=np.single)
            #print(dt)

            pos_fak = f.tell()
            #print(pos_fak)

            fak = []
 
            for n in x:
                f.seek(pos_fak + 4*(n-1))
                fak.append(np.frombuffer(f.read(4),dtype=np.single))
                
            f.seek(pos_fak+lvek[0]*4)
            position = f.tell()
    # the length of the sensors is calculated:
            RecordCount = round((filesize-position)/lvek[0]/2)
            #print(RecordCount)

            raw_ts_int = np.empty([len(x),RecordCount])
           # print(raw_ts_int)
        
            #print("Type:",type(raw_ts_int))
            #print(raw_ts_int.shape)

            for n in range(RecordCount):

                for m, sensor_ in enumerate(x):

                    f.seek(position+2*sensor_+(lvek[0]*2)*n-2)
                    raw_ts_int[m,n] = np.frombuffer(f.read(2),dtype=np.int16)*fak[m]

            global   df
            df = pd.DataFrame(raw_ts_int)

            #print(df.transpose())
            #merged_df=df.merge(df)
                    
    f.close()


In [6]:
%%time
import os
import pandas as pd

sensor = (1, 4, 5, 6, 7, 8, 9, 10, 11, 12, 53, 54, 56, 57, 59, 60, 61, 62, 63, 64)

data_dir = parent_dir
output_dir = processed_data_dir

if not os.path.exists(output_dir):
    os.makedirs(output_dir)

for root, dirs, files in os.walk(data_dir):
    for folder_name in dirs:
        if not folder_name.startswith("Yaw"):
            continue
        
        print(f"Processing data in {folder_name}...")
        
        folder_path = os.path.join(root, folder_name)
        output_folder_path = os.path.join(output_dir, folder_name)
        os.makedirs(output_folder_path, exist_ok=True)
        
        os.chdir(folder_path)
        
        file_list = glob.glob("*.int")
        
        mean_df = pd.DataFrame()
        min_df = pd.DataFrame()
        max_df = pd.DataFrame()
        median_df = pd.DataFrame()
        std_df = pd.DataFrame()
        q1_df = pd.DataFrame()
        q3_df = pd.DataFrame()
        
        for file_path in file_list:
            read_int(file_path, sensor) 
            data = df.transpose()
    
            mean_df = pd.concat([mean_df, data.mean()], axis=1)
            min_df = pd.concat([min_df, data.min()], axis=1)
            max_df = pd.concat([max_df, data.max()], axis=1)
            median_df = pd.concat([median_df, data.median()], axis=1)
            std_df = pd.concat([std_df, data.std()], axis=1)
            q1_df = pd.concat([q1_df, data.quantile(0.25)], axis=1)
            q3_df = pd.concat([q3_df, data.quantile(0.75)], axis=1)
    
        mean_df = mean_df.transpose().reset_index(drop=True)
        min_df = min_df.transpose().reset_index(drop=True)
        max_df = max_df.transpose().reset_index(drop=True)
        median_df = median_df.transpose().reset_index(drop=True)
        std_df = std_df.transpose().reset_index(drop=True)
        q1_df = q1_df.transpose().reset_index(drop=True)
        q3_df = q3_df.transpose().reset_index(drop=True)
        
        mean_df.columns = ['wind_speed', 'yaw_error', 'pitch_angle_b1', 'pitch_angle_b2', 'pitch_angle_b3', 'rotorposition', 'generator_speed', 'rotor_speed', 'generator_torque', 'power', 'acceleration_CROSS', 'acceleration_THRUST', 'tower_deflection_Y', 'tower_deflection_Z', 'tip_deflection_flap_V1', 'tip_deflection_flap_V2', 'tip_deflection_flap_V3', 'tip_deflection_edge_V1', 'tip_deflection_edge_V2', 'tip_deflection_edge_V3']
        min_df.columns = mean_df.columns
        max_df.columns = mean_df.columns
        median_df.columns = mean_df.columns
        std_df.columns = mean_df.columns
        q1_df.columns = mean_df.columns
        q3_df.columns = mean_df.columns
    
        mean_df.to_csv(os.path.join(output_folder_path, f"{folder_name}_mean.csv"), index=False)
        min_df.to_csv(os.path.join(output_folder_path, f"{folder_name}_min.csv"), index=False)
        max_df.to_csv(os.path.join(output_folder_path, f"{folder_name}_max.csv"), index=False)
        median_df.to_csv(os.path.join(output_folder_path, f"{folder_name}_median.csv"), index=False)
        std_df.to_csv(os.path.join(output_folder_path, f"{folder_name}_std.csv"), index=False)
        q1_df.to_csv(os.path.join(output_folder_path, f"{folder_name}_q1.csv"), index=False)
        q3_df.to_csv(os.path.join(output_folder_path, f"{folder_name}_q3.csv"), index=False)

print("Processing done for all folders.")

column_names = {
    '0': 'wind_speed',
    '1': 'yaw_error',
    '2': 'pitch_angle_b1',
    '3': 'pitch_angle_b2',
    '4': 'pitch_angle_b3',
    '5': 'rotorposition',
    '6': 'generator_speed',
    '7': 'rotor_speed',
    '8': 'generator_torque',
    '9': 'power',
    '10': 'acceleration_CROSS',
    '11': 'acceleration_THRUST',
    '12': 'tower_deflection_Y',
    '13': 'tower_deflection_Z',
    '14': 'tip_deflection_flap_V1',
    '15': 'tip_deflection_flap_V2',
    '16': 'tip_deflection_flap_V3',
    '17': 'tip_deflection_edge_V1',
    '18': 'tip_deflection_edge_V2',
    '19': 'tip_deflection_edge_V3',
}

for folder_name in os.listdir(output_dir):
    folder_path = os.path.join(output_dir, folder_name)

    if not os.path.isdir(folder_path) or not folder_name.startswith("Yaw"):
        continue

    csv_files = [f for f in os.listdir(folder_path) if f.endswith('.csv')]

    for file_name in csv_files:
        file_path = os.path.join(folder_path, file_name)
        df = pd.read_csv(file_path)

        df.rename(columns=column_names, inplace=True)

        new_file_name = os.path.splitext(file_name)[0]
        new_file_path = os.path.join(folder_path, new_file_name + '.parquet')

        df.to_parquet(new_file_path, index=False)

print("Column names changed and files saved as Parquet.")


Processing data in Yaw01...
Processing data in Yaw02...
Processing data in Yaw03...
Processing data in Yaw04...
Processing data in Yaw05...
Processing data in Yaw-11...
Processing data in Yaw-12...
Processing data in Yaw-13...
Processing data in Yaw-14...
Processing data in Yaw-15...
Processing data in Yaw-16...
Processing data in Yaw-17...
Processing data in Yaw-18...
Processing data in Yaw-19...
Processing data in Yaw-20...
Processing data in Yaw06...
Processing data in Yaw07...
Processing data in Yaw08...
Processing data in Yaw09...
Processing data in Yaw10...
Processing data in Yaw11...
Processing data in Yaw-06...
Processing data in Yaw-07...
Processing data in Yaw-08...
Processing data in Yaw-09...
Processing data in Yaw-10...
Processing data in Yaw12...
Processing data in Yaw13...
Processing data in Yaw14...
Processing data in Yaw15...
Processing data in Yaw16...
Processing data in Yaw17...
Processing data in Yaw18...
Processing data in Yaw19...
Processing data in Yaw20...
Proce

ValueError: Length mismatch: Expected axis has 0 elements, new values have 20 elements

## Extracting features from raw data and assign columns names

%%time
import os
import pandas as pd

sensor = (1, 4, 5, 6, 7, 8, 9, 10, 11, 12, 53, 54, 56, 57, 59, 60, 61, 62, 63, 64)

data_dir = parent_dir
output_dir = processed_data_dir

if not os.path.exists(output_dir):
    os.makedirs(output_dir)

for root, dirs, files in os.walk(data_dir):
    for folder_name in dirs:
        print(f"Processing data in {folder_name}...")
        
        folder_path = os.path.join(root, folder_name)
        output_folder_path = os.path.join(output_dir, folder_name)
        os.makedirs(output_folder_path, exist_ok=True)
        
        os.chdir(folder_path)
        
        file_list = glob.glob("*.int")
        
        mean_df = pd.DataFrame()
        min_df = pd.DataFrame()
        max_df = pd.DataFrame()
        median_df = pd.DataFrame()
        std_df = pd.DataFrame()
        q1_df = pd.DataFrame()
        q3_df = pd.DataFrame()
        
        for file_path in file_list:
            read_int(file_path, sensor) 
            data = df.transpose()
    
            mean_df = pd.concat([mean_df, data.mean()], axis=1)
            min_df = pd.concat([min_df, data.min()], axis=1)
            max_df = pd.concat([max_df, data.max()], axis=1)
            median_df = pd.concat([median_df, data.median()], axis=1)
            std_df = pd.concat([std_df, data.std()], axis=1)
            q1_df = pd.concat([q1_df, data.quantile(0.25)], axis=1)
            q3_df = pd.concat([q3_df, data.quantile(0.75)], axis=1)
    
        mean_df = mean_df.transpose().reset_index(drop=True)
        min_df = min_df.transpose().reset_index(drop=True)
        max_df = max_df.transpose().reset_index(drop=True)
        median_df = median_df.transpose().reset_index(drop=True)
        std_df = std_df.transpose().reset_index(drop=True)
        q1_df = q1_df.transpose().reset_index(drop=True)
        q3_df = q3_df.transpose().reset_index(drop=True)
        
        mean_df.columns = ['wind_speed', 'yaw_error', 'pitch_angle_b1', 'pitch_angle_b2', 'pitch_angle_b3', 'rotorposition', 'generator_speed', 'rotor_speed', 'generator_torque', 'power', 'acceleration_CROSS', 'acceleration_THRUST', 'tower_deflection_Y', 'tower_deflection_Z', 'tip_deflection_flap_V1', 'tip_deflection_flap_V2', 'tip_deflection_flap_V3', 'tip_deflection_edge_V1', 'tip_deflection_edge_V2', 'tip_deflection_edge_V3']
        min_df.columns = mean_df.columns
        max_df.columns = mean_df.columns
        median_df.columns = mean_df.columns
        std_df.columns = mean_df.columns
        q1_df.columns = mean_df.columns
        q3_df.columns = mean_df.columns
    
        mean_df.to_csv(os.path.join(output_folder_path, f"{folder_name}_mean.csv"), index=False)
        min_df.to_csv(os.path.join(output_folder_path, f"{folder_name}_min.csv"), index=False)
        max_df.to_csv(os.path.join(output_folder_path, f"{folder_name}_max.csv"), index=False)
        median_df.to_csv(os.path.join(output_folder_path, f"{folder_name}_median.csv"), index=False)
        std_df.to_csv(os.path.join(output_folder_path, f"{folder_name}_std.csv"), index=False)
        q1_df.to_csv(os.path.join(output_folder_path, f"{folder_name}_q1.csv"), index=False)
        q3_df.to_csv(os.path.join(output_folder_path, f"{folder_name}_q3.csv"), index=False)

print("Processing done for all folders.")

column_names = {
    '0': 'wind_speed',
    '1': 'yaw_error',
    '2': 'pitch_angle_b1',
    '3': 'pitch_angle_b2',
    '4': 'pitch_angle_b3',
    '5': 'rotorposition',
    '6': 'generator_speed',
    '7': 'rotor_speed',
    '8': 'generator_torque',
    '9': 'power',
    '10': 'acceleration_CROSS',
    '11': 'acceleration_THRUST',
    '12': 'tower_deflection_Y',
    '13': 'tower_deflection_Z',
    '14': 'tip_deflection_flap_V1',
    '15': 'tip_deflection_flap_V2',
    '16': 'tip_deflection_flap_V3',
    '17': 'tip_deflection_edge_V1',
    '18': 'tip_deflection_edge_V2',
    '19': 'tip_deflection_edge_V3',
}

for folder_name in os.listdir(output_dir):
    folder_path = os.path.join(output_dir, folder_name)

    if not os.path.isdir(folder_path):
        continue

    csv_files = [f for f in os.listdir(folder_path) if f.endswith('.csv')]

    for file_name in csv_files:
        file_path = os.path.join(folder_path, file_name)
        df = pd.read_csv(file_path)

        df.rename(columns=column_names, inplace=True)

        new_file_name = os.path.splitext(file_name)[0]
        new_file_path = os.path.join(folder_path, new_file_name + '.parquet')

        df.to_parquet(new_file_path, index=False)

print("Column names changed and files saved as Parquet.")



import os
import pandas as pd

data_dir = parent_dir
output_dir = processed_data_dir

column_names = {
    '0': 'wind_speed',
    '1': 'yaw_error',
    '2': 'pitch_angle_b1',
    '3': 'pitch_angle_b2',
    '4': 'pitch_angle_b3',
    '5': 'rotorposition',
    '6': 'generator_speed',
    '7': 'rotor_speed',
    '8': 'generator_torque',
    '9': 'power',
    '10': 'acceleration_CROSS',
    '11': 'acceleration_THRUST',
    '12': 'tower_deflection_Y',
    '13': 'tower_deflection_Z',
    '14': 'tip_deflection_flap_V1',
    '15': 'tip_deflection_flap_V2',
    '16': 'tip_deflection_flap_V3',
    '17': 'tip_deflection_edge_V1',
    '18': 'tip_deflection_edge_V2',
    '19': 'tip_deflection_edge_V3'
}

if not os.path.exists(output_dir):
    os.makedirs(output_dir)

for root, dirs, files in os.walk(data_dir):
    for folder_name in dirs:
        print(f"Processing data in {folder_name}...")
        
        folder_path = os.path.join(root, folder_name)
        output_folder_path = os.path.join(output_dir, folder_name)
        os.makedirs(output_folder_path, exist_ok=True)
        
        os.chdir(folder_path)
        
        file_list = glob.glob("*.int")
        
        for file_path in file_list:
            read_int(file_path, sensor) 
            data = df.transpose()
            data.columns = column_names.values()
    
            output_file_path = os.path.join(output_folder_path, f"{file_path}.csv")
            data.to_csv(output_file_path, index=False)
        
print("Processing done for all folders.")


## Exporting extracted data to required dir

In [7]:
import pandas as pd
import os

def concatenate_csv_files(directory):
    os.chdir(directory)

    csv_files = [file for file in os.listdir() if file.endswith('.csv')]

    combined_data = pd.DataFrame()

    for file in csv_files:
        aggregation_type = file.split('.')[0].split('_')[-1]
        df = pd.read_csv(file)
        renamed_columns = [f"{column}_{aggregation_type}" for column in df.columns]
        df.columns = renamed_columns
        combined_data = pd.concat([combined_data, df], axis=1)

    sorted_columns = sorted(combined_data.columns)
    combined_data = combined_data[sorted_columns]

    directory_name = os.path.basename(directory)

    output_file_name = f'{directory_name}_combined_data.csv'
    output_directory = os.path.join(processed_data_dir, 'Concatenated_Yaws')
    os.makedirs(output_directory, exist_ok=True)
    output_file_path = os.path.join(output_directory, output_file_name)
    combined_data.to_csv(output_file_path, index=False)

    print(f"The data from {len(csv_files)} CSV files in directory {directory} has been concatenated, rearranged, and exported to {output_file_path}.")


base_directory = processed_data_dir

yaw_directories = [os.path.join(base_directory, directory) for directory in os.listdir(base_directory) if directory.startswith('Yaw')]

for directory in yaw_directories:
    concatenate_csv_files(directory)


The data from 7 CSV files in directory D:\processed_data\Yaw01 has been concatenated, rearranged, and exported to D:\processed_data\Concatenated_Yaws\Yaw01_combined_data.csv.
The data from 7 CSV files in directory D:\processed_data\Yaw02 has been concatenated, rearranged, and exported to D:\processed_data\Concatenated_Yaws\Yaw02_combined_data.csv.
The data from 7 CSV files in directory D:\processed_data\Yaw03 has been concatenated, rearranged, and exported to D:\processed_data\Concatenated_Yaws\Yaw03_combined_data.csv.
The data from 7 CSV files in directory D:\processed_data\Yaw04 has been concatenated, rearranged, and exported to D:\processed_data\Concatenated_Yaws\Yaw04_combined_data.csv.
The data from 7 CSV files in directory D:\processed_data\Yaw05 has been concatenated, rearranged, and exported to D:\processed_data\Concatenated_Yaws\Yaw05_combined_data.csv.
The data from 7 CSV files in directory D:\processed_data\Yaw-11 has been concatenated, rearranged, and exported to D:\process

import pandas as pd
import os

def concatenate_csv_files(directory):
    os.chdir(directory)

    csv_files = [file for file in os.listdir() if file.endswith('.csv')]

    combined_data = pd.DataFrame()

    for file in csv_files:
        aggregation_type = file.split('.')[0].split('_')[-1]
        df = pd.read_csv(file)
        renamed_columns = [f"{column}_{aggregation_type}" for column in df.columns]
        df.columns = renamed_columns
        combined_data = pd.concat([combined_data, df], axis=1)

    sorted_columns = sorted(combined_data.columns)
    combined_data = combined_data[sorted_columns]

    directory_name = os.path.basename(directory)

    output_file_name = f'{directory_name}_combined_data.csv'
    output_directory = concate_yaw_dir
    os.makedirs(output_directory, exist_ok=True)
    output_file_path = os.path.join(output_directory, output_file_name)
    combined_data.to_csv(output_file_path, index=False)

    print(f"The data from {len(csv_files)} CSV files in directory {directory} has been concatenated, rearranged, and exported to {output_file_path}.")


base_directory = processed_data_dir

yaw_directories = [os.path.join(base_directory, directory) for directory in os.listdir(base_directory) if directory.startswith('Yaw')]

for directory in yaw_directories:
    concatenate_csv_files(directory)
